In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os, sys

#### Open csv-files with OpenFace analysis results

In [15]:
def get_one_video(file_name: str):
    with open(file_name) as f:
        video_df = pd.read_csv(file_name, sep=',')
        return video_df[[' timestamp', ' confidence', ' pose_Rx', ' pose_Ry', ' pose_Rz']]

#### Build graph for sentence with negation boundaries

In [38]:
def build_and_save_graph(graph_name: str, df: pd.DataFrame, neg_annotations=None):
    plt.plot(df[' timestamp'], df[' pose_Ry'])
    
    for annot in neg_annotations: 
        plt.vlines(x=annot[0] / 1000, ymin=min(df[' pose_Ry']), ymax=max(df[' pose_Ry']), colors='red', linestyles='dashed')
        plt.vlines(x=annot[1] / 1000, ymin=min(df[' pose_Ry']), ymax=max(df[' pose_Ry']), colors='red', linestyles='dashed')
    
    path = f'../new_extracted/annot_cut_3/graphs/{graph_name}.png'
    plt.savefig(path)
    plt.close()

#### Add offsets that we used when cutted our fragments

In [40]:
def get_processed_name(path):
    path_dirs = path.split('/')
    return '.'+'/'.join(path_dirs[:-1]) + '/processed/' + path_dirs[-1] + '.csv'


def get_neg_annots(annotations, s_offset, end_offset):
    annots_with_offset = []
    
    for annot in annotations:
        annots_with_offset.append((annot[0] + s_offset + 80, annot[1] + end_offset - 80))
    
    return annots_with_offset


def get_graph_for_one_fragment(row):
    path = row['path']
    annotations, s_offset, end_offset = row['annotations']
    
    processed_path = get_processed_name(path)
    neg_annotations = get_neg_annots(annotations, s_offset, end_offset)
    
    df = get_one_video(processed_path)
    graph_name = processed_path.split('/')[-1].replace('.csv', '')
    
    build_and_save_graph(graph_name, df, neg_annotations)

In [41]:
annot_df = pd.read_csv('../video_annotations.csv')
annot_df.drop(columns=['Unnamed: 0'], inplace=True)
annot_df.annotations = annot_df.annotations.apply(eval)

In [42]:
annot_df

,path,annotations
0,./new_extracted/new_cut/new_fragments/RSLM-cr1...,"([(2440, 2680, )], -80, 80)"
1,./negation VK 21.03.2022/RSLM-cr4-s13_NE_6,"([(0, 720, hs)], -80, 80)"
2,./negation VK 21.03.2022/RSLM-cr5-s22_NE_10,"([(200, 520, hs)], -80, 80)"
3,./negation VK 21.03.2022/RSLM-m5-s40-d-std_NE_20,"([(520, 1080, hs)], 80, 240)"
4,./negation VK 21.03.2022/RSLM-m5-s40-d-std_NEG_0,"([(1320, 1680, hs)], -80, 80)"
...,...,...
67,./negation VK 21.03.2022/RSLN-n5-s25-d-std_NET...,"([(1650, 2975, hs)], -80, 80)"
68,./negation VK 21.03.2022/RSLN-n5-s25-d-std_NET...,"([(5500, 6625, hs)], -80, 80)"
69,./negation VK 21.03.2022/RSLN-n5-s25-d-std_NET...,"([(3975, 4925, hs)], -80, 80)"
70,./negation VK 21.03.2022/RSLN-n5-s25-d-std_NET...,"([(1600, 3000, hs)], -80, 80)"


In [43]:
annot_df.apply(get_graph_for_one_fragment, axis=1)

C:\anaconda\lib\site-packages\matplotlib\cbook\__init__.py:1402: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x[:, None]
C:\anaconda\lib\site-packages\matplotlib\axes\_base.py:276: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x = x[:, np.newaxis]
C:\anaconda\lib\site-packages\matplotlib\axes\_base.py:278: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  y = y[:, np.newaxis]


0     None
1     None
2     None
3     None
4     None
      ... 
67    None
68    None
69    None
70    None
71    None
Length: 72, dtype: object